In [1]:
from datareader.datareader import DatasetReader
from retriever.retriever_topk import TopkRetriever

obj = DatasetReader(embed = True, embedding_model_path='/data/lhb/huggingface/model/tokenizer/Qwen2.5-7B-instruct')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
obj.embedding_with_LM()


200it [07:30,  2.25s/it]


## 读取测试数据

In [ ]:
import pickle

with open('/data/lhb/test-openicl-0.1.8/EHR_Base/results/readmission_prediction/randseed42/bge_embed_test42.pkl', 'rb') as file:
    test_embedding_ds = pickle.load(file)
    
test_embedding_ds[0]

{'embed': array([ 0.03450291, -0.0017248 , -0.04685855, ...,  0.01775567,
        -0.03873008, -0.03169431], dtype=float32),
 'id': 0,
 'raw_text': "The patient is Female and was diagnosed with various diseases such as Grand mal status、Pneumonitis due to inhalation of food or vomitus、Acidosis, and in response to these diagnoses, the doctor prescribed medications such as Divalproex Sodium 500 MG Enteric Coated Tablet [Depakote]、phenytoin sodium 100 MG Extended Release Oral Capsule、200 ACTUAT Albuterol 0.09 MG/ACTUAT / Ipratropium Bromide 0.018 MG/ACTUAT Metered Dose Inhaler [Combivent]、topiramate 25 MG Oral Tablet [Topamax]、Amoxicillin 875 MG / Clavulanate 125 MG Oral Tablet、0、heparin sodium, porcine 5000 UNT/ML Injectable Solution、1 ML lorazepam 2 MG/ML Cartridge、5 ML valproic acid 100 MG/ML Injection [Depacon]、folic acid 5 MG/ML Injectable Solution、50 ML sodium chloride 9 MG/ML Injection、1000 ML glucose 50 MG/ML / sodium chloride 4.5 MG/ML Injection、Phenytoin sodium 50 MG/ML Injectabl

In [25]:
import numpy as np

embed = test_embedding_ds[0]['embed']  # 由于 search 的参数需要是 二维的，这里增加一个维度
embed = np.float32(embed)

In [ ]:
embed = embed[0].tolist()
embed = np.float32([embed])

In [22]:
embed

array([[ 0.00046571, -0.00426205, -0.00105008, ...,  0.00514582,
         0.00326655, -0.00246059]], dtype=float32)

In [7]:
import faiss

train_index = faiss.read_index('/data/lhb/test-openicl-0.1.8/EHR_Base/faiss_index/faiss_index_qwen7B_train_readmission.index')

In [26]:
train_index.search(embed, 3)

(array([[0.99606556, 0.9957382 , 0.99548733]], dtype=float32),
 array([[22936, 46523, 15185]]))

## random_idx

In [5]:
import random
random.seed(36)

def generate_random_lists(num_lists):
    """
    生成一个二维列表，每个元素是一个包含3个随机整数的一维列表。
    随机整数范围是0到4000。

    参数:
        num_lists (int): 二维列表中一维列表的数量。

    返回:
        list: 生成的二维列表。
    """
    result = []
    for _ in range(num_lists):
        # 生成一个包含3个随机整数的一维列表
        random_list = [random.randint(0, 47179) for _ in range(3)]
        # 将一维列表添加到二维列表中
        result.append(random_list)
    return result

# 示例用法
num_lists = 200  # 假设我们想要生成5个一维列表
random_2d_list = generate_random_lists(num_lists)
print(random_2d_list)

[[21541, 3825, 1366], [18643, 5150, 26], [32933, 41090, 11564], [16229, 17656, 42752], [28337, 23888, 36705], [17569, 35571, 5095], [37759, 26728, 32113], [25556, 22645, 25928], [15945, 25694, 10987], [8090, 33694, 32596], [45980, 13339, 34353], [19288, 12089, 32225], [46233, 38882, 21268], [46745, 35995, 11042], [207, 38920, 25262], [32413, 9084, 14261], [27370, 24371, 13642], [32413, 16789, 40059], [2394, 15472, 41073], [33241, 30299, 42580], [33279, 19396, 29666], [5538, 15389, 19781], [13318, 37210, 10979], [11632, 4173, 15704], [24499, 28427, 46788], [35713, 1062, 40947], [24934, 22813, 32036], [11150, 35146, 36942], [16627, 5575, 6480], [42555, 29781, 13757], [14662, 45219, 23287], [9671, 9189, 11873], [35406, 41957, 5605], [31054, 18130, 18348], [19340, 171, 35406], [19668, 45284, 36689], [3908, 19585, 2237], [32616, 5240, 24596], [45013, 7695, 19662], [26954, 24616, 12193], [17746, 21189, 13138], [9042, 18494, 29567], [28140, 7005, 41186], [11459, 7314, 8266], [30442, 44737, 20

In [ ]:
import json

with open('rand_idx.json', 'w', encoding='utf-8') as file:
    json.dump(random_2d_list, file, ensure_ascii=False, indent=4)

In [9]:
"""
    进行 case study, 读取 idx
"""

import json

with open('/data/lhb/test-openicl-0.1.8/EHR_Base/results/mortality_prediction/test_ds[:200]/topk/bge_embed/topk_ice_idx.json', 'r', encoding='utf-8') as file:
    bge_idx = json.load(file)

with open('/data/lhb/test-openicl-0.1.8/EHR_Base/results/mortality_prediction/test_ds[:200]/topk/qwen7B_embed/top3_qwenEmbed_ice_idx.json', 'r', encoding='utf-8') as file:
    qwen_idx = json.load(file)

In [4]:
"""
    进行 case study, 读取 label
"""

import json

with open('/data/lhb/test-openicl-0.1.8/EHR_Base/results/mortality_prediction/test_ds[:200]/topk/bge_embed/topk_res_deepseek_r1.json', 'r', encoding='utf-8') as file:
    bge_label = json.load(file)

with open('/data/lhb/test-openicl-0.1.8/EHR_Base/results/mortality_prediction/test_ds[:200]/topk/qwen7B_embed/top3_res_qwenEmbed_deepseek_r1.json', 'r', encoding='utf-8') as file:
    qwen_label = json.load(file)

In [17]:
bge_label[131], qwen_label[130]

({'id': 131,
  'logits': [1.769980788230896, 23.134546279907227],
  'prediction': [0.07107064444099966, 0.9289293555590004],
  'label': 'died',
  'prediction_softmax': [5.266078684229569e-10, 1.0]},
 {'id': 131,
  'logits': [24.925935745239258, 15.259057998657227],
  'prediction_softmax': [0.9999366998672485, 6.334332283586264e-05],
  'label': 'died'})

In [18]:
bge_idx[131], qwen_idx[131]

([16839, 28337, 31039], [31039, 12644, 15648])

In [16]:
for bge, qwen in zip(bge_label[80:], qwen_label[79:]): # 寻找 qwen deepseekr1 作对的，bge 做错的 case
    if bge['logits'][0] < bge['logits'][1] and qwen['logits'][0] > qwen['logits'][1] and qwen['label'] == 'died':  # 80, 
    # if bge['logits'][0] > bge['logits'][1] and qwen['logits'][0] < qwen['logits'][1] and qwen['label'] == 'survived':  # 无
        print(bge['id'])

87
92
130
131
142
150
151
161
176
189
190
196


In [14]:
with open('/data/lhb/test-openicl-0.1.8/EHR_Base/cone3_qwen_idx.json', 'r', encoding='utf-8') as file:
    bge_idx = json.load(file)

len(bge_idx)

100